An underground railway system is keeping track of customer travel times between different stations. They are using this data to calculate the average time it takes to travel from one station to another.

Implement the UndergroundSystem class:

    void checkIn(int id, string stationName, int t)
        A customer with a card ID equal to id, checks in at the station stationName at time t.
        A customer can only be checked into one place at a time.
    void checkOut(int id, string stationName, int t)
        A customer with a card ID equal to id, checks out from the station stationName at time t.
    double getAverageTime(string startStation, string endStation)
        Returns the average time it takes to travel from startStation to endStation.
        The average time is computed from all the previous traveling times from startStation to endStation that happened directly, meaning a check in at startStation followed by a check out from endStation.
        The time it takes to travel from startStation to endStation may be different from the time it takes to travel from endStation to startStation.
        There will be at least one customer that has traveled from startStation to endStation before getAverageTime is called.

You may assume all calls to the checkIn and checkOut methods are consistent. If a customer checks in at time t1 then checks out at time t2, then t1 < t2. All events happen in chronological order.


# Example 1:

 ## Input
["UndergroundSystem","checkIn","checkIn","checkIn","checkOut","checkOut","checkOut","getAverageTime","getAverageTime","checkIn","getAverageTime","checkOut","getAverageTime"]
[[],[45,"Leyton",3],[32,"Paradise",8],[27,"Leyton",10],[45,"Waterloo",15],[27,"Waterloo",20],[32,"Cambridge",22],["Paradise","Cambridge"],["Leyton","Waterloo"],[10,"Leyton",24],["Leyton","Waterloo"],[10,"Waterloo",38],["Leyton","Waterloo"]]

## Output
[null,null,null,null,null,null,null,14.00000,11.00000,null,11.00000,null,12.00000]

## Explanation
    UndergroundSystem undergroundSystem = new UndergroundSystem();

    undergroundSystem.checkIn(45, "Leyton", 3);

    undergroundSystem.checkIn(32, "Paradise", 8);

    undergroundSystem.checkIn(27, "Leyton", 10);

    undergroundSystem.checkOut(45, "Waterloo", 15);  // Customer 45 "Leyton" -> "Waterloo" in 15-3 = 12

    undergroundSystem.checkOut(27, "Waterloo", 20);  // Customer 27 "Leyton" -> "Waterloo" in 20-10 = 10

    undergroundSystem.checkOut(32, "Cambridge", 22); // Customer 32 "Paradise" -> "Cambridge" in 22-8 = 14

    undergroundSystem.getAverageTime("Paradise", "Cambridge"); // return 14.00000. One trip "Paradise" -> "Cambridge", (14) / 1 = 14

    undergroundSystem.getAverageTime("Leyton", "Waterloo");    // return 11.00000. Two trips "Leyton" -> "Waterloo", (10 + 12) / 2 = 11

    undergroundSystem.checkIn(10, "Leyton", 24);

    undergroundSystem.getAverageTime("Leyton", "Waterloo");    // return 11.00000

    undergroundSystem.checkOut(10, "Waterloo", 38);  // Customer 10 "Leyton" -> "Waterloo" in 38-24 = 14

    undergroundSystem.getAverageTime("Leyton", "Waterloo");    // return 12.00000. Three trips "Leyton" -> "Waterloo", (10 + 12 + 14) / 3 = 12


Constraints:

    1 <= id, t <= 106
    1 <= stationName.length, startStation.length, endStation.length <= 10
    All strings consist of uppercase and lowercase English letters and digits.
    There will be at most 2 * 104 calls in total to checkIn, checkOut, and getAverageTime.
    Answers within 10-5 of the actual value will be accepted.



In [4]:
class UndergroundSystem:
    
    #Our plan of attack will be to keep a ledger tracking the passengers that will
    #save when they join and then delete when they checkout, and when we remove them
    #from the ledger, we will record the time differnce.

    #In a separate ledger tracking stations, once we delete a passenger, we will
    #update this ledger with a journey time, and a tracker of the number of
    #successful check-in/check-outs to generate our average per journey.
    #Each key in this ledger will not be per station, but instead of will 
    #generate a new entry once a passenger leaves that will be constituted
    #by the check-in station and the checkout station.
    def __init__(self):
        #this will track our passengers as they check-in/check-out
        #the scheme for this dictionary will be:
        #   [passenger ID] = (station name, travel time)
        self.passLedge = {}
        
        #this will track our intrastation times
        #the scheme for this dictionary will be:
        #   [tuple of two station names] = [total minutes for this combo, number of journeys]
        self.stationsLedge = {}

    def checkIn(self, id: int, stationName: str, t: int) -> None:
        #making sure we don't already have the passenger checked in,
        #we save in our passenger ledger the station paired with the time
        #we store this in a tuple because we wont be changing this pair
        if id not in self.passLedge.keys():
            self.passLedge[id] = (stationName, t)
        else: 
            print("chief, this non-gendered dude is already checked in, oof")


    def checkOut(self, id: int, stationName: str, t: int) -> None:
        #we check to see if the person checking out is in our ledger
        if id not in self.passLedge.keys():
            print('who u trying 2 check out there buddy? capser???!')
        
        #otherwise, in our station pairs ledger, we create a new key
        #called from our passenger's check in and our passed to
        #station name so that it's always start -> finish.
        else:
            #we create a new station-pair in our station ledger that
            #equals (checkin station, checkout station)
            stationJourneyPair = (self.passLedge[id][0],stationName)

            #if our station-pair isn't in our station ledger then we
            #create a new key and pass it a list of the difference of 
            #the checkout - checkin times and a journey count of 1
            if stationJourneyPair not in self.stationsLedge.keys():
                self.stationsLedge[stationJourneyPair] = [t - self.passLedge[id][1], 1]
            
            #if we do already have this station pair in the ledger, we will update it
            #by adding the new travel time and incrementing the journey counter by one
            elif stationJourneyPair in self.stationsLedge.keys():
                self.stationsLedge[stationJourneyPair][0] += (t - self.passLedge[id][1])
                self.stationsLedge[stationJourneyPair][1] += 1

            #now we must clean our passenger ledger since our checkin/checkout
            #journey is complete
            del self.passLedge[id]
            

    def getAverageTime(self, startStation: str, endStation: str) -> float:
        stationsKey = (startStation, endStation)
        if stationsKey not in self.stationsLedge:
            print('ya let me just average NOTHING, NERD!!')
        else:
            return((self.stationsLedge[stationsKey][0])/self.stationsLedge[stationsKey][1])
        


# Your UndergroundSystem object will be instantiated and called as such:
# obj = UndergroundSystem()
# obj.checkIn(id,stationName,t)
# obj.checkOut(id,stationName,t)
# param_3 = obj.getAverageTime(startStation,endStation)

In [5]:
temp = UndergroundSystem()
temp.checkIn(1,'deeznuts',1)
temp.checkIn(2,'hehe town', 1)
temp.checkOut(1,'fuckuville', 100)
temp.checkOut(2,'oh god help', 50)
temp.checkIn(3,'deeznuts', 3)
temp.checkOut(3,'fuckuville', 30)
temp.getAverageTime('deeznuts','fuckuville')

63.0